In [3]:
import pandas as pd
import numpy as np

# Creating dataset

## Straight movement

In [2]:
df = pd.DataFrame(columns=["x","x_vel","y","y_vel"]) #,"x0","x1","y0","y1"])

In [3]:
# df.append({"test1": 1, "Test2" : 2}, ignore_index=True)

In [6]:
dt = 0.01

def add(row, df):
    """Docstring"""
    d = dict()
    cols = list(df.columns)
    for i in np.arange(len(cols)):
        d[cols[i]] = row[i]
    return df.append(d, ignore_index=True)

def add_rnd(df):
    x_vel = np.sign(np.random.rand() - 0.5) * np.random.rand() * 0.1
    y_vel = np.sign(np.random.rand() - 0.5) * np.random.rand() * 0.1
    x = np.random.rand()
    y = np.random.rand()
    return add([x,x_vel,y,y_vel],df) #,0,1,0,1], df)

def add_next(df):
    x,x_vel,y,y_vel = list(df.iloc[-1])
    return add([x+x_vel,x_vel,y+y_vel,y_vel], df)

In [6]:
df = pd.DataFrame(columns=["x","x_vel","y","y_vel"]) #,"x0","x1","y0","y1"])
for i in range(100000):
    if i % 10 == 0: df = add_rnd(df)
    else: df = add_next(df)

KeyboardInterrupt: 

In [6]:
df.shape

(100000, 4)

In [9]:
df.to_csv('sim_data2.csv', index=False)

### Vectorized

## Straight movement with borders

Assume collisions to be fully elastic and border corners are at (0,0), (0,1), (1,0), (1,1)

In [7]:
def add_check_borders(c, c_vel):
    c2, c_vel2 = c + c_vel, c_vel
    if c2 > 1:
        c2 = 1 - np.abs(1-c2)
        c_vel2 = -c_vel
    elif c2 < 0:
        c2 = np.abs(c2)
        c_vel2 = -c_vel
    return c2, c_vel2

def add_borders_next(df):
    x,x_vel,y,y_vel = list(df.iloc[-1])
    x2, xv2 = add_check_borders(x, x_vel)
    y2, yv2 = add_check_borders(y, y_vel)
    return add([x2,xv2,y2,yv2], df)

In [8]:
df = pd.DataFrame(columns=["x","x_vel","y","y_vel"]) #,"x0","x1","y0","y1"])
for i in range(20):
    if i % 20 == 0: df = add([0.93, 0.9, 0.92, 0.9], df) #df = add_rnd(df)
    else: df = add_borders_next(df)

df.head()

,x,x_vel,y,y_vel
0,0.93,0.9,0.92,0.9
1,0.17,-0.9,0.18,-0.9
2,0.73,0.9,0.72,0.9
3,0.37,-0.9,0.38,-0.9
4,0.53,0.9,0.52,0.9


In [9]:
import matplotlib.pyplot as plt

#plt.plot(df.x, df.y)
plt.scatter(df.x, df.y, s=1)
plt.figure()
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 0 Axes>

In [10]:
df = pd.DataFrame(columns=["x","x_vel","y","y_vel"]) #,"x0","x1","y0","y1"])
for i in range(20000):
    if i % 20 == 0: df = add_rnd(df)
    else: df = add_borders_next(df)
        
df.to_csv('sim_data_borders.csv', index=False)

## Bounces only

In [11]:
df = pd.DataFrame(columns=["x","x_vel","y","y_vel"])
for i in range(2000):
    if i % 2 == 0:
        x_vel = np.random.rand() * np.sign(np.random.rand()-0.5)
        y_vel = np.random.rand() * np.sign(np.random.rand()-0.5)
        if 1:#if np.random.rand() >0.5:
            x = np.random.uniform(low=0, high=np.abs(x_vel) * dt)
            y = np.random.rand()
            if np.sign(x_vel) > 0: x = 1 - x
        else:
            y = np.random.uniform(low=0, high=np.abs(y_vel) * dt)
            x = np.random.rand()
            if np.sign(y_vel) > 0: y = 1 - y
        df = add([x,x_vel,y,y_vel],df)
    else: df = add_borders_next(df)
df.to_csv('bounces2.csv', index=False)

In [12]:
df.head()

,x,x_vel,y,y_vel
0,0.997198,0.322083,0.230437,-0.900816
1,0.999581,-0.322083,0.221429,-0.900816
2,0.002769,-0.366834,0.307852,-0.021624
3,0.000899,0.366834,0.307636,-0.021624
4,0.001572,-0.599076,0.739328,-0.696767


## Series leading up to a bounce

In [6]:
df = pd.DataFrame(columns=["x","x_vel","y","y_vel"])
for i in range(2000):
    if i % 2 == 0:
        x_vel = np.random.rand() * 0.1 #* np.sign(np.random.rand()-0.5) * 0.1
        y_vel = np.random.rand() * np.sign(np.random.rand()-0.5) * 0.1
        if 1:#if np.random.rand() >0.5:
            x = np.random.uniform(low=0, high=np.abs(x_vel) * dt)
            y = np.random.rand()
            if np.sign(x_vel) > 0: x = 1 - x
        else:
            y = np.random.uniform(low=0, high=np.abs(y_vel) * dt)
            x = np.random.rand()
            if np.sign(y_vel) > 0: y = 1 - y
#         df = add([x,x_vel,y,y_vel],df)
        for i in range(3, -1, -1):
            df = add([x-i*x_vel,x_vel,y-i*y_vel,y_vel],df)
    else: 
        df = add_borders_next(df)
df.to_csv('bounces_x_1.csv', index=False)

In [7]:
df.head()

,x,x_vel,y,y_vel
0,0.900505,0.033114,0.388893,-0.057712
1,0.933619,0.033114,0.331182,-0.057712
2,0.966733,0.033114,0.273470,-0.057712
3,0.999847,0.033114,0.215758,-0.057712
4,0.999821,-0.033114,0.215181,-0.057712
